In [1]:
from sklearn.model_selection import GridSearchCV
from pandas import read_csv
from sklearn import svm
import pandas as pd
import numpy as np
import lightgbm as lgb
from sklearn.model_selection import  GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import RandomizedSearchCV
from skopt import BayesSearchCV
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.metrics import roc_auc_score, roc_curve

In [2]:
# Read data
train_data = pd.read_csv("/Users/alex/Desktop/data/6_train_dataset_augmented.csv")
X_train = train_data.drop('is_fraud', axis=1)
y_train = train_data['is_fraud']

test_data = pd.read_csv("/Users/alex/Desktop/data/4_test_dataset.csv")
X_test = test_data.drop('is_fraud', axis=1)
y_test = test_data['is_fraud']

## Training SVM model

In [3]:
# train SVM model
def train_model_svm(X_train, y_train, svm_model):

    # hyperparameter
    param_space = {
        'C': (0.1, 1, 10, 1000),
        'gamma': [0.1, 1, 10]
    }

    random_search = RandomizedSearchCV(
        svm_model,
        param_space,
        n_iter=5,  # Number of parameter settings that are sampled
        cv=3,
        n_jobs=-1,
        random_state=42
    )
    # fit the Bayesian search to your data
    random_search.fit(X_train, y_train)

    # get the best hyperparameters
    best_params = random_search.best_params_

    # print the best hyperparameters found
    print("Best Hyperparameters:", best_params)

    # set the best hyperparameters to the model
    svm_model.set_params(**best_params)

    # train the final svm model on the entire training set
    svm_model.fit(X_train, y_train)

In [4]:
def evaluate_model(model, predX, predY):

    y_pred = model.predict(predX)

    # accuracy
    acc =  accuracy_score(predY, y_pred)
    print("Accuracy: %.2f%%" % (acc * 100))

    # precision
    precision = precision_score(predY, y_pred)
    print("Precision: %.2f" % precision)

    # recall
    recall = recall_score(predY, y_pred)
    print("Recall: %.2f" % recall)

    # f1 Score
    f1 = f1_score(predY, y_pred)
    print("F1 Score: %.2f" % f1)

    # ROC-AUC score
    roc_auc = roc_auc_score(predY, y_pred)
    print("ROC-AUC Score: %.2f" % roc_auc)

In [5]:
# create the LightGBM model
svm_model = svm.SVC()

# train model
train_model_svm(X_train,y_train, svm_model)

Best Hyperparameters: {'gamma': 1, 'C': 1000}


In [6]:
evaluate_model(svm_model,X_test, y_test)

Accuracy: 37.43%
Precision: 0.26
Recall: 0.97
F1 Score: 0.41
ROC-AUC Score: 0.59
